# Packaging an Overlay

This notebook will demonstrate how to package an Overlay. This notebook depends on the previous three notebooks:

2. [Creating a Vivado HLS Core](2-Creating-A-Vivado-HLS-Core.ipynb)
3. [Building a Vivado Bitstream](3-Building-A-Bitstream.ipynb)
4. [Using an HLS core in PYNQ](4-Using-an-HLS-core-in-PYNQ.ipynb)

In this notebook you will generate the following file inside the `tutorial` directory:

- `setup.py`

This will allow you to install your PYNQ Overlays as Python packages using Pip, and distribute them on GitHub (Like this tutorial was delivered)

## Outputs from Previous Steps

The first two critical components of a PYNQ overlay are a `.tcl` script file and a `.bit`. These files should have been created in **[Creating a FPGA Bitstream](2-Creating-A-Bitstream.ipynb)** and with the names `sharedmem.tcl` and `sharedmem.bit`. 

In addition, you will need the `__init__.py` and `sharedmem.py` python files created in **[Using an HLS core in PYNQ](4-Using-an-HLS-core-in-PYNQ.ipynb)**. These files are necessary for communcation between the ARM Processing System and FPGA Programmable Logic.

Verify that the following files are in the `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem` directory of your PYNQ board.
   1. `sharedmem.tcl`
   2. `sharedmem.bit`
   3. `sharedmem.py`
   4. `__init__.py`

In [ ]:
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/sharedmem.tcl
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/sharedmem.bit
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/sharedmem.py
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/__init__.py

## Notebooks Folder

Finally, we need to create a `notebooks` folder and populate its contents. Inside of `notebooks` we need to create a `pictures` folder. 

The following cell will perform this operation:

In [ ]:
!mkdir /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/notebooks
!mkdir /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/notebooks/pictures

You can create your own notebooks inside of this folder, or you can copy the existing demonstration notebook:

In [ ]:
!cp -r /home/xilinx/PYNQ-HLS/pynqhls/sharedmem/notebooks/*.ipynb /home/xilinx/PYNQ-HLS/tutorial/pynqhls/sharedmem/notebooks

## Automating Installation

The next step is to automate the installation using [PIP](https://pip.pypa.io).

In the previous notebook you added your `sharedmemOverlay` class to the Python Search path. By creating a `setup.py` file and running `pip3.6` you will be able to run your overlay from any Python shell without modifying the search path.

To do this, we will create a `setup.py` script in the repository.

You cannot skip this step, but fortunately, we provide working code for you.

### Creating a `setup.py` file

A `setup.py` file is the central part of distributing a python package using PIP. In this step we will create a simple `setup.py` script that executes the following steps:

1. Installs the notebooks folder as a folder with the name `HLS-Tutorial-Output` in the Jupyter Home directory (`/home/xilinx/jupyter_notebooks` on a PYNQ board)
2. Installs the overlay (`sharedmem.bit` file, `sharedmem.tcl` file, `__init__.py`, and `sharedmem.py`) from the previous step into the python `site-packages` folder.

More documentation regarding `setup.py` and setup scripts can be found on the [distutils documentation page](https://docs.python.org/3/distutils/setupscript.html).

Copy and paste the following cell into `setup.py` inside `/home/xilinx/PYNQ-HLS/tutorial/` on your PYNQ board:

In [ ]:
from setuptools import setup
import os

jupyter_dest = '/home/xilinx/jupyter_notebooks'

# Find all of the tutorial notebooks in the notebooks_src path                                                                                                                                                                                     
notebooks_src = 'pynqhls/sharedmem/notebooks/'
notebooks_dest = os.path.join(jupyter_dest, 'HLS-Tutorial-Output')
notebooks = [os.path.join(notebooks_src, f)
             for f in os.listdir(notebooks_src)]

# Find all of the tutorial notebook pictures in the pictures_src path                                                                                                                                                                         
notebooks_pictures_src = os.path.join(notebooks_src, 'pictures')
notebooks_pictures_dest = os.path.join(notebooks_dest, 'pictures')
notebooks_pictures = [os.path.join(notebooks_pictures_src, f)
                      for f in os.listdir(notebooks_pictures_src)]

notebooks.remove(notebooks_pictures_src)

setup(name='pynq-hls-tutorial',
      version='0.1',
      description="A simple package describing how to create a PYNQ \
      bitstream with HLS cores",
      author='Your Name',
      author_email='youremail@yourinstitution.edu',
      license='BSD-3',
      data_files = [(notebooks_dest, notebooks),
                    (notebooks_pictures_dest, notebooks_pictures)],
      package_dir = {'pynqhls_tutorial': './pynqhls'},
      packages=['pynqhls_tutorial', 'pynqhls_tutorial.sharedmem'],
      package_data={'':['*.bit', '*.tcl']},
      install_requires=['pynq'],
      dependency_links=['http://github.com/xilinx/PYNQ.git@v2.0#egg=pynq'],
)


The `setup.py` file starts with a set of variable declarations:

- The variable `jupyter_dest` defines the path of the Jupyter Notebooks directory on the PYNQ board. 
- The variable `notebooks` defines the location of the `.ipynb` notebook files for delivery, and `notebooks_dest` defines the destination. Likewise for `notebooks_pictures_src` and `notebooks_pictures_dest`.

Inside the call to `setup()` we pass the arguments [`data_files`](https://docs.python.org/3.6/distutils/setupscript.html#installing-additional-files), [`packages` and `package_dir`](https://docs.python.org/3.6/distutils/setupscript.html#listing-whole-packages), and [`package_data`](https://docs.python.org/3.6/distutils/setupscript.html#installing-package-data). 

- `data_files` is a list of Relative-Source and Destination-Path tuples used to install non-python files (like notebooks and pictures)
- `packages` is a list of Python packages to install
- `package_dir` is a dictonary mapping package names to directory locations
- `package_data` is a dictionary containing a package and list of relative files that should be included in that package

In order to install this as a PYNQ package using `setup.py` we need to modify all four variables, as shown above

Once you have saved `setup.py` in `/home/xilinx/PYNQ-HLS/tutorial` you will be able to install the IO Overlay you have created on your PYNQ board using the cell below:

In [ ]:
!pip3.6 install /home/xilinx/PYNQ-HLS/tutorial/

When the command has completed reload this notebook and run the following cell. Note how the package name has changed from `pynqhls` to `pynqhls_tutorial` to avoid conflicts in the Python package namespace: 

In [ ]:
from pynqhls_tutorial.sharedmem import sharedmemOverlay
overlay = sharedmemOverlay('sharedmem.bit')

If you are successful, you can re-run the same tests from the previous notebook:

In [ ]:
import numpy as np

A = np.random.randint(-10, 10, size=(10,10))
B = np.random.randint(-10, 10, size=(10,10))
C = np.matmul(A, B)
CHLS = overlay.run(A, B)
np.array_equal(CHLS, C)

## Installing from Git

The steps we have shown above install a Python package from a local directory. To install a Python Package from Git you must provide a `setup.py` file in the root directory. 

These tutorials demonstrate one example of using a `setup.py` file to distribute a PYNQ Pver;ay. The `setup.py` file we use is shown below: 

In [ ]:
from setuptools import setup

import os

jupyter_dest = '/home/xilinx/jupyter_notebooks'
# We provide three tutorials: 
tutorials = ["stream", "sharedmem", "io"]

# Find all of the tutorial notebooks in the tutorials_base path
tutorials_base = 'tutorial/notebooks/'
tutorials_dest_base = os.path.join(jupyter_dest, 'HLS-Tutorial')

data_files = []
# For each tutorial, install the notebooks and pictures for the tutorial.
for tut in tutorials:
    nbsource = os.path.join(tutorials_base, tut)
    notebooks = [os.path.join(nbsource, f) for f in os.listdir(nbsource)]
    nbdest = os.path.join(tutorials_dest_base, tut)
    picsource = os.path.join(nbsource, 'pictures')
    pictures = [os.path.join(picsource, f) for f in os.listdir(picsource)]
    picdest = os.path.join(nbdest, 'pictures')
    notebooks.remove(picsource)
    data_files.append((nbdest, notebooks))
    data_files.append((picdest, pictures))

demo_base = 'pynqhls/'
demo_dest_base = os.path.join(jupyter_dest, 'HLS-Demo')

# For each tutorial, install the demo notebook and pictures
for tut in tutorials:
    nbsource = os.path.join(demo_base, tut, 'notebooks')
    notebooks = [os.path.join(nbsource, f) for f in os.listdir(nbsource)]
    nbdest = os.path.join(demo_dest_base, tut)
    picsource = os.path.join(nbsource, 'pictures')
    pictures = [os.path.join(picsource, f) for f in os.listdir(picsource)]
    picdest = os.path.join(nbdest, 'pictures')
    notebooks.remove(picsource)
    data_files.append((nbdest, notebooks))
    data_files.append((picdest, pictures))

setup(name='pynq-hls',
      version='0.1',
      description="A simple package describing how to create a PYNQ\
            bitstream with HLS cores",
      author='Dustin Richmond',
      author_email='drichmond@eng.ucsd.edu',
      url='https://github.com/drichmond/PYNQ-HLS/',
      license='BSD-3',
      data_files = data_files,
      packages=['pynqhls', 'pynqhls.stream', 'pynqhls.io', 'pynqhls.sharedmem'],
      package_data={'':['*.bit', '*.tcl']},
      install_requires=['pynq'],
      dependency_links=['http://github.com/xilinx/PYNQ.git@v2.0#egg=pynq'],
)
